In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import unicodedata

In [7]:
def askUrl(url):
    resp = requests.get(url,headers = header).text
    soup = BeautifulSoup(resp)
    return soup

In [8]:
def cleanData(text):
    text1 = re.sub('\[(.*?)\]','',text).replace('\n','')
    text = unicodedata.normalize('NFKD',text1)
    return text

In [9]:
def getData(soup):
    tables = soup.select('#mw-content-text > div.mw-parser-output > table')
#mw-content-text > div.mw-parser-output > table:nth-child(21) > tbody > tr:nth-child(2)
#mw-content-text > div.mw-parser-output > table:nth-child(24) > tbody > tr:nth-child(2)
#mw-content-text > div.mw-parser-output > table:nth-child(27) > tbody > tr:nth-child(2) > td:nth-child(2)
    data = []
    for z in tables:
        a = z.select('tbody > tr')
        if len(a[0])>=20:
            for j in range(1,len(a)-1,2):
                while a[j].find(scope="row")==None:
                    j+=1
                flightNo = a[j].find(scope="row").get_text()
                flightNo = cleanData(flightNo)
                #print(flightNo)
                # Date and time (UTC)
                Dt0 = a[j].select('td:nth-child(2)')[0].get_text()
                Dt = cleanData(Dt0)
                #print(Dt)
                #Version,booster
                Vb0 = a[j].select('td:nth-child(3)')[0].get_text()
                Vb = cleanData(Vb0)
                #print(Vb)
                # Launch site
                Ls0 = a[j].select('td:nth-child(4)')[0].get_text()
                Ls = cleanData(Ls0)
                #print(Ls)
                #Payload
                Pl0 = a[j].select('td:nth-child(5)')[0].get_text()
                Pl = cleanData(Pl0)
                #print(Pl)
                #Payload mass
                Pm0 = a[j].select('td:nth-child(6)')[0].get_text()
                Pm = cleanData(Pm0)
                #print(Pm)

                Orbit0 = a[j].select('td:nth-child(7)')[0].get_text()
                Orbit = cleanData(Orbit0)
                #print(Orbit)
                #Customer	
                Cus0 = a[j].select('td:nth-child(8)')[0].get_text()
                Cus = cleanData(Cus0)
                #print(Cus)
                #Launch outcome
                Lo0 = a[j].select('td:nth-child(9)')[0].get_text()
                Lo = cleanData(Lo0)
                #print(Lo)
                #Booster landing
                Bl0 = a[j].select('td:nth-child(10)')[0].get_text()
                Bl = cleanData(Bl0)
                #print(Bl)
                tmp = [flightNo,Dt,Vb,Ls,Pl,Pm,Orbit,Cus,Lo,Bl]
                #print(tmp)
                data.append(tmp)
    return data

In [10]:
if __name__=="__main__":
    header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

    url = ["https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)","https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"]

    cols = ['flightNo','Date and time (UTC)','Version, Booster','Launch site','Payload',
           'Payload mass','Orbit','Customer','Launch outcome','Booster landing']
    dataF =[]
    for u in url:
        resp = askUrl(u)
        df =pd.DataFrame(data = getData(resp),columns=cols)
        #data = getData(resp)
        df.to_csv('./F9wiki.csv',mode='a',index=False)

In [11]:
df = pd.read_csv('./F9wiki.csv')

In [12]:
df

,flightNo,Date and time (UTC),"Version, Booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,"4 June 2010,18:45",F9 v1.0B0003,"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,No payload (excl. Dragon Mass),LEO,SpaceX,Success,Failure(parachute)
1,2,"8 December 2010,15:43",F9 v1.0B0004,"CCAFS,SLC-40",SpaceX COTS Demo Flight 1(Dragon C101),Unknown (excl. Dragon Mass),LEO (ISS),NASA (COTS)various others,Success,Failure(parachute)
2,3,"22 May 2012,07:44",F9 v1.0B0005,"CCAFS,SLC-40",SpaceX COTS Demo Flight 2(Dragon C102),"525 kg (1,157 lb) (excl. Dragon mass)",LEO (ISS),NASA (COTS),Success,No attempt
3,4,"8 October 2012,00:35",F9 v1.0B0006,"CCAFS,SLC-40",SpaceX CRS-1(Dragon C103),"4,700 kg (10,400 lb) (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,No attempt
4,5,"1 March 2013,15:10",F9 v1.0B0007,"CCAFS,SLC-40",SpaceX CRS-2(Dragon C104),"4,877 kg (10,752 lb) (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...
173,166,22 July 202217:39,F9 B5♺B1071.4,"VSFB,SLC-4E",Starlink Group 3-2 (46 satellites),"~14,100 kg (31,100 lb)",SSO,SpaceX,Success,Success(drone ship)
174,167,24 July 202213:38,F9 B5 ♺B1062.8,"KSC,LC-39A",Starlink Group 4-25 (53 satellites),"~16,250 kg (35,830 lb)",LEO,SpaceX,Success,Success(drone ship)
175,168,4 August 202223:08,F9 B5 ♺B1052.6,"CCSFS,SLC-40",Danuri (Korea Pathfinder Lunar Orbiter),"~679 kg (1,497 lb)",Ballistic Lunar Transfer (BLT),KARI,Success,Success(drone ship)
176,169,10 August 202202:14,F9 B5♺B1073.3,"KSC,LC-39A",Starlink Group 4-26 (52 satellites),"~16,000 kg (35,000 lb)",LEO,SpaceX,Success,Success(drone ship)
